In [1]:
import pandas as pd
from agentic.enhancer import AgenticEnhancer
import json
from termcolor import colored

In [2]:
def agentic_enhancer(original_prompt):
    
    agentic = AgenticEnhancer(original_prompt)
    agentic_answer = agentic.execute_workflow()

    original_prompt = agentic_answer.get("original_prompt")
    original_prompt_answer = agentic_answer.get("original_prompt_answer")
    original_prompt_lin_probs = agentic_answer.get("original_prompt_lin_probs")
    enhanced_prompt = agentic_answer.get("enhanced_prompt")
    final_prompt_answer = agentic_answer.get("final_prompt_answer")
    final_prompt_lin_probs = agentic_answer.get("final_prompt_lin_probs")
    
    final_result = {
        "original_prompt": original_prompt,
        "original_prompt_answer": original_prompt_answer,
        "original_prompt_lin_probs": original_prompt_lin_probs,
        "enhanced_prompt": enhanced_prompt,
        "final_prompt_answer": final_prompt_answer,
        "final_prompt_lin_probs": final_prompt_lin_probs,
    }
    
    return final_result

In [8]:
# with open('data/prompts.json', 'r') as f:
#     original_prompt = json.load(f)

# df = pd.DataFrame(
#     {
#         "original_prompt": original_prompt,
#         "original_prompt_answer": [None]*len(original_prompt),
#         "original_prompt_lin_probs": [None]*len(original_prompt),
#         "enhanced_prompt": [None]*len(original_prompt),
#         "final_prompt_answer": [None]*len(original_prompt),
#         "final_prompt_lin_probs": [None]*len(original_prompt),
#     }
# )

# display(df.head(5))




# Test with a single prompt
test_prompt = "As a highly respected international trade policy expert, provide a comprehensive and insightful analysis of the current tariff dynamics between the United States and Canada. Begin your response with a concise executive summary highlighting key insights and recommendations. Then, explain the historical context and economic rationale behind the implementation of these tariffs, clearly describing the strategic goals and economic motivations involved. Next, analyze in detail at least three major industries notably affected by these tariffs—such as automotive, agriculture, or technology—and explain the immediate impacts, as well as secondary or indirect economic consequences observed in each sector. Include recent, post-2022 data, statistics, and concrete real-world examples to illustrate how tariffs have influenced factors like trade volume, pricing, employment, consumer prices, and broader economic performance. Critically evaluate both short-term and long-term benefits and drawbacks of these tariffs from multiple perspectives, specifically addressing the effects on businesses, consumers, workers, and governmental policies in both Canada and the United States. Following your evaluation, present two clearly defined future scenarios: one in which tariffs remain unchanged, and another scenario where tariffs are significantly reduced or removed altogether. For each scenario, provide thoughtful projections for economic outcomes over the next five years, using logical reasoning and evidence-based forecasting. Finally, conclude by proposing three to five actionable and evidence-supported policy recommendations that policymakers could realistically implement in the short term to optimize bilateral economic growth, reduce negative impacts, and strengthen the Canada-U.S. trade relationship. Structure your response clearly, use precise economic terminology, and ensure it remains engaging and accessible throughout."
enhancer = AgenticEnhancer(test_prompt)
final_result = enhancer.execute_workflow()

# Create DataFrame with single result
df = pd.DataFrame([{
    "original_prompt": test_prompt,
    "original_prompt_answer": final_result.get('original_prompt_answer', ''),
    "original_prompt_lin_probs": final_result.get('original_prompt_lin_probs', 0.0),
    "enhanced_prompt": final_result.get('enhanced_prompt', ''),
    "final_prompt_answer": final_result.get('final_prompt_answer', ''),
    "final_prompt_lin_probs": final_result.get('final_prompt_lin_probs', 0.0),
}])

# Set display options for better readability
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Display the DataFrame
display(df)



Category Determined:  {query="As a highly respected international trade policy expert, provide a comprehensive and insightful analysis of the current tariff dynamics between the United States and Canada. Begin your response with a concise executive summary highlighting key insights and recommendations. Then, explain the historical context and economic rationale behind the implementation of these tariffs, clearly describing the strategic goals and economic motivations involved. Next, analyze in detail at least three major industries notably affected by these tariffs—such as automotive, agriculture, or technology—and explain the immediate impacts, as well as secondary or indirect economic consequences observed in each sector. Include recent, post-2022 data, statistics, and concrete real-world examples to illustrate how tariffs have influenced factors like trade volume, pricing, employment, consumer prices, and broader economic performance. Critically evaluate both short-term and long-ter

,original_prompt,original_prompt_answer,original_prompt_lin_probs,enhanced_prompt,final_prompt_answer,final_prompt_lin_probs
0,"As a highly respected international trade policy expert, provide a comprehensive and insightful analysis of the current tariff dynamics between the United States and Canada. Begin your response with a concise executive summary highlighting key insights and recommendations. Then, explain the historical context and economic rationale behind the implementation of these tariffs, clearly describing the strategic goals and economic motivations involved. Next, analyze in detail at least three major industries notably affected by these tariffs—such as automotive, agriculture, or technology—and explain the immediate impacts, as well as secondary or indirect economic consequences observed in each sector. Include recent, post-2022 data, statistics, and concrete real-world examples to illustrate how tariffs have influenced factors like trade volume, pricing, employment, consumer prices, and broader economic performance. Critically evaluate both short-term and long-term benefits and drawbacks of these tariffs from multiple perspectives, specifically addressing the effects on businesses, consumers, workers, and governmental policies in both Canada and the United States. Following your evaluation, present two clearly defined future scenarios: one in which tariffs remain unchanged, and another scenario where tariffs are significantly reduced or removed altogether. For each scenario, provide thoughtful projections for economic outcomes over the next five years, using logical reasoning and evidence-based forecasting. Finally, conclude by proposing three to five actionable and evidence-supported policy recommendations that policymakers could realistically implement in the short term to optimize bilateral economic growth, reduce negative impacts, and strengthen the Canada-U.S. trade relationship. Structure your response clearly, use precise economic terminology, and ensure it remains engaging and accessible throughout.",,0.0,"**Enhanced Prompt for Analysis of U.S.-Canada Tariff Dynamics:**\n\n**Role**: As a highly respected international trade policy expert, your task is to provide a comprehensive and insightful analysis of the current tariff dynamics between the United States and Canada.\n\n**Instructions**: Your response should be structured clearly and include the following components:\n\n1. **Executive Summary**: \n - Begin with a concise summary that highlights key insights and actionable recommendations regarding the current tariff situation.\n\n2. **Historical Context**: \n - Provide an explanation of the historical background and economic rationale for the implementation of these tariffs. \n - Detail the strategic goals and motivations behind their establishment.\n\n3. **Industry Impact Analysis**: \n - Analyze at least three major industries notably affected by these tariffs, such as automotive, agriculture, and technology. \n - Discuss both the immediate impacts and the secondary economic consequences observed in each sector, supported by recent data and real-world examples from post-2022.\n\n4. **Evaluation of Effects**: \n - Critically evaluate the short-term and long-term benefits and drawbacks of these tariffs from multiple perspectives, specifically addressing the effects on businesses, consumers, workers, and governmental policies in both Canada and the United States.\n\n5. **Future Scenarios**: \n - Present two clearly defined future scenarios: \n - One where tariffs remain unchanged.\n - Another where tariffs are significantly reduced or removed altogether. \n - For each scenario, provide thoughtful projections for economic outcomes over the next five years, utilizing logical reasoning and evidence-based forecasting.\n\n6. **Policy Recommendations**: \n - Conclude with three to five actionable and evidence-supported policy recommendations that policymakers could realistically implement in the short term

In [ ]:
for i, row in df.iterrows():
    original_prompt = row['original_prompt']
    final_result = agentic_enhancer(original_prompt)
    
    df.at[i, 'original_prompt_answer'] = final_result['original_prompt_answer']
    df.at[i, 'original_prompt_lin_probs'] = final_result['original_prompt_lin_probs']
    df.at[i, 'enhanced_prompt'] = final_result['enhanced_prompt']
    df.at[i, 'final_prompt_answer'] = final_result['final_prompt_answer']
    df.at[i, 'final_prompt_lin_probs'] = final_result['final_prompt_lin_probs']
    
display(df)

In [6]:
df.to_csv('results/enhancement_results_1.csv', index=False)

#### --------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('results/enhancement_results_1.csv').iloc[:, 1:]
df['original_prompt_lin_probs'] = df['original_prompt_lin_probs'].apply(lambda x: round(x, 6))

mask = df['original_prompt_lin_probs'] < 0.20
df.loc[mask, 'original_prompt_lin_probs'] = 0.50

display(df.head(5))

In [ ]:
print(df.describe())

In [ ]:
# 1. Calculate and visualize average improvement percentage
def plot_improvement_distribution():
    # Calculate improvement percentage
    df['improvement_percentage'] = ((df['final_prompt_lin_probs'] - df['original_prompt_lin_probs']) 
                                  / df['original_prompt_lin_probs'] * 100)
    
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df, x='improvement_percentage', fill=True, color='skyblue')
    plt.axvline(df['improvement_percentage'].mean(), color='r', linestyle='--', 
                label=f'Mean Improvement: {df["improvement_percentage"].mean():.1f}%')
    
    plt.title('Distribution of Prompt Enhancement Improvements')
    plt.xlabel('Improvement Percentage')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Average improvement: {df['improvement_percentage'].mean():.1f}%")
    print(f"Maximum improvement: {df['improvement_percentage'].max():.1f}%")
    print(f"Minimum improvement: {df['improvement_percentage'].min():.1f}%")

# 2. Compare original vs enhanced confidence scores
def plot_confidence_comparison():
    plt.figure(figsize=(12, 6))
    indices = range(len(df))
    width = 0.35
    
    plt.bar(indices, df['original_prompt_lin_probs'], width, 
            label='Original Confidence', color='lightblue')
    plt.bar([i + width for i in indices], df['final_prompt_lin_probs'], width,
            label='Enhanced Confidence', color='darkblue')
    
    plt.xlabel('Prompt Index')
    plt.ylabel('Model Confidence (Linear Probability)')
    plt.title('Original vs Enhanced Prompt Confidence')
    plt.legend()
    plt.xticks([i + width/2 for i in indices], indices)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_improvement_distribution()
plot_confidence_comparison()